In [4]:
from werkzeug.wrappers import Request, Response
from flask import Flask,render_template,send_file,make_response,request
import json
import pandas as pd

import numpy as np
from sklearn import ensemble, model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from lime.lime_tabular import LimeTabularExplainer

app = Flask(__name__)
print(dir(json))

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
dataset  = pd.read_csv(url, names=['sepal length', 'sepal width', 'petal length', 'petal width','target'])


def index_containing_substring(the_list, substring):
    for i, s in enumerate(the_list):
        if substring in s:
              return i
    return -1

def reduce_dim(df):
    variables = ['sepal length', 'sepal width', 'petal length', 'petal width']
    x = df.loc[:, variables].values
    y = df.loc[:,['target']].values
    x = StandardScaler().fit_transform(x)
    x = pd.DataFrame(x)
    pca = PCA()
    x_pca = np.array(pca.fit_transform(x))
    display(pca.explained_variance_ratio_)
    return pd.DataFrame(x_pca[:,0:2],columns=['x','y'])

@app.route('/box')
def boxplot():
    ret = pd.DataFrame({'x':np.random.randint(-1000, high=1000, size=1000),
                        'y':np.random.randint(-1, high=2000, size=1000),
                        'z':np.random.randint(-1000, high=2000, size=1000),
                       })
    ret=ret.set_index('x')
    resp = make_response(ret.to_csv())
    resp.headers["Content-Disposition"] = "attachment; filename=export.csv"
    resp.headers["Content-Type"] = "text/csv"
    return resp




@app.route('/favicon.ico')
def icon():
    return None

@app.route('/doc_indx_<docs>')
def get_documents(docs):
    return [1,2,3,4,5]

@app.route('/')
def route():
    return render_template('index.html')

@app.route('/get_dataset_<dataset>')
def load_dataset(dataset):
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
    df = pd.read_csv(url, names=['sepal length', 'sepal width', 'petal length', 'petal width','target'])

    df = reduce_dim(df)
    df['label'] = 1
    df['color'] = 'blue'

    
    display(df)
    return df.to_json()

@app.route('/elaborate_dataset')
def evaluate():
    j=json.loads(request.args.get('dataset'))
    data = pd.DataFrame(j)
    #dataset['target'] = j
    data = dataset.select_dtypes(['number'])
    display(data)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
    data, j)

    regressor = ensemble.RandomForestRegressor()
    regressor.fit(X_train, y_train)
    
    display(X_train)
    display(regressor.predict(X_test))
    explainer = LimeTabularExplainer(X_train.to_numpy(),
                                     training_labels =['sepal length', 'sepal width', 'petal length', 'petal width'],
                                     feature_names=['sepal length', 'sepal width', 'petal length', 'petal width'],
                                     mode='regression')

    X_test = X_test.to_numpy()
    # Now explain a prediction
    exp = explainer.explain_instance(X_test[25], regressor.predict,
            num_features=4)
    
    exp.save_to_file("templates/explanation_test.html")
    
    f = open("templates/explanation_test.html", "r",encoding="utf8")
    contents = f.readlines()
    f.close()
    
    index = index_containing_substring(contents, ".templateSettings.interpolate ")
    del contents[index]
    
    f = open("templates/explanation_test.html", "w",encoding="utf8")
    contents = "".join(contents)
    f.write(contents)
    f.close()
    
    evaluation = {'src':'explanation_test'}
    return json.dumps(evaluation).encode("utf8")

@app.route('/<page>')
def rout(page):
    return render_template(page+'.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9001, app,reloader_interval = 5000)

['JSONDecodeError', 'JSONDecoder', 'JSONEncoder', '__all__', '__author__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_default_decoder', '_default_encoder', 'codecs', 'decoder', 'detect_encoding', 'dump', 'dumps', 'encoder', 'load', 'loads', 'scanner']


 * Running on http://localhost:9001/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Mar/2021 17:08:22] "GET /test_histogram HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:08:22] "GET /static/Histogram.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:08:22] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:08:51] "GET /test_histogram HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:08:51] "GET /static/Histogram.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:08:51] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:09:01] "GET /test_boxplot HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:09:01] "GET /static/Boxplot.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Mar/2021 17:09:01] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:15:54] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:15:55] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:15:55] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:16:40] "GET /test_performance_whee

127.0.0.1 - - [09/Mar/2021 17:34:18] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:34:47] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:34:47] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:34:47] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:00] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:00] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:00] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:08] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:08] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:08] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:15] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:15] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:35:15] "GET /box HTTP/1.1" 200 -
127.0.0.1

127.0.0.1 - - [09/Mar/2021 17:49:54] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:49:59] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:49:59] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:49:59] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:50:24] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:50:24] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:50:25] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:51:00] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:51:00] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:51:00] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:51:01] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 17:51:01] "GET /static/Performance_wheel.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Mar/2021 17:51:01] "GET /box HTTP/1.1" 200 -
127.0.0.1

127.0.0.1 - - [09/Mar/2021 18:10:01] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:10] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:10] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:10] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:35] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:35] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:35] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:41] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:41] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:41] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:43] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:10:43] "GET /static/Performance_wheel.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Mar/2021 18:10:43] "GET /box HTTP/1.1" 200 -
127.0.0.1

127.0.0.1 - - [09/Mar/2021 18:16:58] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:17:08] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:17:08] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:17:08] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:17:59] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:17:59] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:17:59] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:18:01] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:18:01] "GET /static/Performance_wheel.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Mar/2021 18:18:01] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:18:15] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:18:15] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:18:16] "GET /box HTTP/1.1" 200 -
127.0.0.1

127.0.0.1 - - [09/Mar/2021 18:28:17] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:28] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:28] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:28] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:32] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:32] "GET /static/Performance_wheel.js HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:32] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:36] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:36] "GET /static/Performance_wheel.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Mar/2021 18:28:37] "GET /box HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:39] "GET /test_performance_wheel HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 18:28:39] "GET /static/Performance_wheel.js HTTP/1.1" 304 -
127.0.0.1 - - [09/Mar/2021 18:28:39] "GET /box HTTP/1.1" 200 -


In [3]:
#Your imports
from flask import Flask, render_template
from bokeh.embed import components
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, CustomJS

app = Flask(__name__)


@app.route('/')
def homepage():
    title = 'home'
    x = [x for x in range(-10, 10)]
    y = x


    source=ColumnDataSource(data=dict(x=x, y=y))
    TOOLS = "box_select,lasso_select,help"

    p = figure(tools = TOOLS)
    p.scatter('x', 'y', source=source)

    p2 = figure(tools = TOOLS)
    p2.scatter('x', 'y', source=source)
    
    source.selected.js_on_change('indices', CustomJS(code = """
    console.log(cb_obj.indices);
    """))
    def update_plot(attrname, old, new):
        pass
    
    source.selected.on_change('indices', update_plot)

    script, div = components(row(p,p2))
    script2, div2 = components(p)

    return render_template('embed.html', title = title, script = script,
    div = div)

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9001, app,reloader_interval = 5000)

 * Running on http://localhost:9001/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Mar/2021 13:18:57] "GET /test_histogram HTTP/1.1" 404 -


In [2]:
from flask import Flask, render_template

from bokeh.client import pull_session
from bokeh.embed import server_session

app = Flask(__name__)

@app.route('/', methods=['GET'])
def bkapp_page():

    # pull a new session from a running Bokeh server
    with pull_session(url="http://localhost:5006/sliders") as session:

        # update or customize that session
        session.document.roots[0].children[1].title.text = "Special Sliders For A Specific User!"

        # generate a script to load the customized session
        script = server_session(session_id=session.id, url='http://localhost:5006/sliders')

        # use the script in the rendered page
        return render_template("embed.html", script=script, template="Flask")

if __name__ == '__main__':
    app.run(port=8080)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cristian\Miniconda3\envs\visualizationEnviroment\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-55db18eb32c5>", line 3, in <module>
    from bokeh.client import pull_session
  File "C:\Users\Cristian\Miniconda3\envs\visualizationEnviroment\lib\site-packages\bokeh\client\__init__.py", line 32, in <module>
    from .session import (
  File "C:\Users\Cristian\Miniconda3\envs\visualizationEnviroment\lib\site-packages\bokeh\client\session.py", line 38, in <module>
    from ..document import Document
  File "C:\Users\Cristian\Miniconda3\envs\visualizationEnviroment\lib\site-packages\bokeh\document\__init__.py", line 35, in <module>
    from .document import DEFAULT_TITLE ; DEFAULT_TITLE
  File "C:\Users\Cristian\Miniconda3\envs\visualizationEnviroment\lib\site-packages\bokeh\document\document.py", line 48, in <module>
    from ..core.

TypeError: object of type 'NoneType' has no len()

mokup both backend and frontend

design document

draft of framework with pipeline from start to finish
